##Step 4: Run Classification Models

In [1]:
% matplotlib qt

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import exec.utilgen as utgen
import exec.model_framework.utilmodel as utmdl
import exec.model_framework.model as mdl
import exec.data_framework.data_pipeline as dtp
import cProfile
import itertools

% reload_ext autoreload
% autoreload 2


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


D:\SoftwareWork\PythonProjects\Miniconda\envs\datascience1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


###Data

In [2]:
version = 5
dataPath = os.path.join('data', 'data_derived', 'train1.csv')
dataTPath = os.path.join('data', 'data_derived', 'train2.csv')
data = pd.read_csv(dataPath, index_col=0)
dataT = pd.read_csv(dataTPath, index_col=0)
with utgen.printOptions(precision=2):
    print('---------- Original Data (Train) -----------')
    display(data.sample(n=5))
    display(data.describe())
    print('---------- Original Data (Test) -----------')
    display(dataT.sample(n=5))
    display(dataT.describe())
    print('---------- Clean Data (Train) -----------')
    data, dataT = dtp.featuresPipelineTrainTest(data, dataT, version=version)
    display(data.sample(n=5))
    display(data.describe())
    print('---------- Clean Data (Test) -----------')
    display(dataT.sample(n=5))
    display(dataT.describe())

---------- Original Data (Train) -----------

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
82,1,3,"Sheerlinck, Mr. Jan Baptist",male,29.00,0,0,345779,9.50,NaN,S
7,0,1,"McCarthy, Mr. Timothy J",male,54.00,0,0,17463,51.86,E46,S
146,0,2,"Nicholls, Mr. Joseph Charles",male,19.00,1,1,C.A. 33112,36.75,NaN,S
559,1,1,"Taussig, Mrs. Emil (Tillie Mandelbaum)",female,39.00,1,1,110413,79.65,E67,S
353,0,3,"Elias, Mr. Tannous",male,15.00,1,1,2695,7.23,NaN,C


,Survived,Pclass,Age,SibSp,Parch,Fare
count,623.00,623.00,499.00,623.00,623.00,623.00
mean,0.37,2.29,29.81,0.49,0.36,32.79
std,0.48,0.85,14.44,0.97,0.75,51.63
min,0.00,1.00,0.42,0.00,0.00,0.00
25%,0.00,1.00,21.00,0.00,0.00,7.92
50%,0.00,3.00,28.00,0.00,0.00,14.45
75%,1.00,3.00,39.00,1.00,0.00,30.50
max,1.00,3.00,74.00,8.00,5.00,512.33


---------- Original Data (Test) -----------

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
286,0,3,"Stankovic, Mr. Ivan",male,33.00,0,0,349239,8.66,NaN,C
142,1,3,"Nysten, Miss. Anna Sofia",female,22.00,0,0,347081,7.75,NaN,S
419,0,2,"Matthews, Mr. William John",male,30.00,0,0,28228,13.00,NaN,S
346,1,2,"Brown, Miss. Amelia ""Mildred""",female,24.00,0,0,248733,13.00,F33,S
41,0,3,"Ahlin, Mrs. Johan (Johanna Persdotter Larsson)",female,40.00,1,0,7546,9.47,NaN,S


...

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
PassengerId,,,,,,,,,
707,1,2,45.00,0,0,13.50,0,1,1
511,1,3,29.00,0,0,7.75,0,0,0
531,1,2,2.00,1,1,26.00,0,1,1
651,0,3,28.96,0,0,7.90,0,1,0
668,0,3,28.96,0,0,7.78,0,1,0


,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
count,623.00,623.00,623.00,623.00,623.00,623.00,623.00,623.00,623.00
mean,0.37,2.29,29.81,0.40,0.35,29.84,0.18,0.74,0.33
std,0.48,0.85,13.15,0.63,0.69,36.66,0.39,0.44,0.47
min,0.00,1.00,0.42,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,1.00,22.00,0.00,0.00,7.92,0.00,0.00,0.00
50%,0.00,3.00,28.96,0.00,0.00,14.45,0.00,1.00,0.00
75%,1.00,3.00,37.00,1.00,0.00,30.50,0.00,1.00,1.00
max,1.00,3.00,71.58,2.00,3.00,163.53,1.00,1.00,1.00


---------- Clean Data (Test) -----------

,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
PassengerId,,,,,,,,,
618,0,3,26.00,1,0,16.10,0,1,1
26,1,3,38.00,1,3,31.39,0,1,1
301,1,3,28.96,0,0,7.75,0,0,1
137,1,1,19.00,0,2,26.28,0,1,1
679,0,3,43.00,1,3,46.90,0,1,1


,Survived,Pclass,Age,SibSp,Parch,Fare,EmbarkedC,EmbarkedS,Female
count,268.00,268.00,268.00,268.00,268.00,268.00,268.00,268.00,268.00
mean,0.41,2.35,29.23,0.41,0.39,29.05,0.20,0.70,0.41
std,0.49,0.81,13.46,0.66,0.77,32.38,0.40,0.46,0.49
min,0.00,1.00,0.67,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,2.00,21.00,0.00,0.00,7.90,0.00,0.00,0.00
50%,0.00,3.00,28.96,0.00,0.00,14.48,0.00,1.00,0.00
75%,1.00,3.00,36.00,1.00,0.00,33.63,0.00,1.00,1.00
max,1.00,3.00,71.58,2.00,3.00,163.53,1.00,1.00,1.00


###Logistic

In [61]:
# cProfile.run('modelLR.fit(data)')

In [7]:
modelLR = mdl.Logistic(fit_intercept=False, scale=True, C=10)
modelLR.fit(data)
modelLR.predict(dataT)
modelLR.printPlotSummary()


****** LOGISTIC ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.50    0.13
1    0.08    0.29

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.46    0.13
1    0.09    0.32

-----Coefficients-----

Pclass      -0.80
Age         -0.49
SibSp       -0.09
Parch       -0.15
Fare         0.13
EmbarkedC    0.01
EmbarkedS   -0.08
Female       1.32
dtype: float64

###Logistic Ridge

In [5]:
modelLRRidgeCV = mdl.LogisticRidgeCV(fit_intercept=False, scale=True, Cs=np.exp(np.arange(-5, 6, 1)))
modelLRRidgeCV.fit(data)
modelLRRidgeCV.predict(dataT)
modelLRRidgeCV.printPlotSummary()

****** LOGISTIC RIDGE ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.46    0.13
1    0.08    0.33

-----Coefficients-----

Pclass      -0.74
Age         -0.39
Fare         0.08
EmbarkedC    0.01
EmbarkedS   -0.09
Female       1.24
SibSpYes    -0.02
ParchYes    -0.01
dtype: float64

-----Ridge CV multiplier-----

Ridge Multiplier = 0.37

###Logistic Best Subset

In [8]:
modelLRBestK = mdl.LogisticBestSubset(fit_intercept=False, scale=True, k=5)
modelLRBestK.fit(data)
modelLRBestK.predict(dataT)
modelLRBestK.printPlotSummary()

****** LOGISTIC KBEST ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.48    0.15
1    0.09    0.28

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.46    0.14
1    0.09    0.31

-----Coefficients-----

Pclass      -0.55
Parch       -0.07
Fare         0.17
EmbarkedC    0.08
Female       1.30
dtype: float64

###Logistic GAM

In [37]:
modelGAM = mdl.LogisticGAM(scale=True, fit_intercept=False, n_splines=15, lam=5)
modelGAM.fit(data)
modelGAM.predict(dataT)
modelGAM.printPlotSummary()

****** LOGISTIC GAM ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.50    0.10
1    0.11    0.29

-----Statistics-----

Feature Function   Data Type      Num Splines   Spline Order  Linear Fit  Lambda     P > x      Sig. Code 
================== ============== ============= ============= =========== ========== ========== ==========
Pclass_________    numerical      15            3             False       5.0        3.27e-03   **        
Age____________    numerical      15            3             False       5.0        1.21e-03   **        
SibSp__________    numerical      15            3             False       5.0        9.70e-02   .         
Parch__________    numerical      15            3             False       5.0        3.68e-01             
Fare___________    numerical      15            3             False       5.0        2.40e-02   *         
EmbarkedC______    numerical      15            3             False       5.0        9.69e-01             
EmbarkedS______    numerical      15            3             False       5.0        6.97e-01             
Female_________    numerical      15 

Significance codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

###Logistic Local (Linear proxy)

In [25]:
modelLocal = mdl.LogisticLinearLocal(scale=True, reg_type='ll', bw=1.)
modelLocal.fit(data)
modelLocal.predict(dataT)
modelLocal.printPlotSummary()


****** LOGISTIC LOCAL (LINEAR PROXY) ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

precision	 (IS / CV / OOS): 0.90 / 0.78 (0.07) / 0.80

-----Confusion (IS)-----

   0-pred  1-pred
0    0.59    0.03
1    0.09    0.29

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.52    0.07
1    0.11    0.30

### Logistic Bayes

In [40]:
modelLogisticBayesian = mdl.LogisticBayesian(scale=True, featuresSd=10, nsamplesFit=200, nsamplesPredict=200,
                                             mcmc=True, nsampleTune=200, discardTuned=True,
                                             samplerStep=None, samplerInit='advi')
modelLogisticBayesian.fit(data)
modelLogisticBayesian.predict(dataT)
modelLogisticBayesian.printPlotSummary()
modelLogisticBayesian.plotTrace()


  0%|          | 0/10000 [00:00<?, ?it/s]

Average Loss = 639.55:   4%|▍         | 411/10000 [00:00<00:02, 4104.99it/s]

Average Loss = 608.83:   8%|▊         | 790/10000 [00:00<00:02, 4003.97it/s]

Average Loss = 578.09:  12%|█▏        | 1181/10000 [00:00<00:02, 3971.89it/s]

Average Loss = 541.76:  16%|█▌        | 1579/10000 [00:00<00:02, 3972.52it/s]

...

Average Loss = 280.9:  95%|█████████▍| 9458/10000 [00:02<00:00, 3967.75it/s] 

Average Loss = 280.02:  99%|█████████▉| 9876/10000 [00:02<00:00, 4029.03it/s]

Average Loss = 279.93: 100%|██████████| 10000/10000 [00:02<00:00, 4071.23it/s]


Finished [100%]: Average Loss = 279.95


-----Performance-----
accuracy	 (IS / CV / OOS): 0.78 / 0.78 / 0.79
accproba	 (IS / CV / OOS): 0.69 / 0.68 / 0.69
logproba	 (IS / CV / OOS): -0.48 / -0.50 / -0.50
aucproba	 (IS / CV / OOS): 0.85 / 0.85 / 0.85
recall	 (IS / CV / OOS): 0.77 / 0.77 / 0.80
precision	 (IS / CV / OOS): 0.68 / 0.67 / 0.71

-----Confusion (IS)-----
   0-pred  1-pred
0    0.49    0.13
1    0.09    0.29
-----Confusion (OOS)-----
   0-pred  1-pred
0    0.46    0.13
1    0.08    0.32



###KNN

In [71]:
modelKNN = mdl.KNN(scale=True, n_neighbors=10, weights='uniform')
modelKNN.fit(data)
modelKNN.predict(dataT)
modelKNN.printPlotSummary()

****** KNN ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

precision	 (IS / CV / OOS): 0.84 / 0.76 (0.11) / 0.86

-----Confusion (IS)-----

   0-pred  1-pred
0    0.58    0.05
1    0.13    0.24

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.54    0.05
1    0.12    0.29

###KNN CV

In [11]:
modelKNNCV = mdl.KNNCV(cv=5, scale=True, weights='uniform')
modelKNNCV.fit(data)
modelKNNCV.predict(dataT)
modelKNNCV.printPlotSummary()

*DEPRECIATED*  Use genModelCV instead 

****** KNN CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

...

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.51    0.08
1    0.12    0.29

-----Best CV Parameters-----

n_neighbors = 10.00

...with the score = 0.79

In [9]:
KNNCV2 = mdl.genModelCV(mdl.KNN, grid={'n_neighbors': (5, 10, 20, 40)})
modelKNNCV2 = KNNCV2(cv=5, scale=True, weights='uniform')
modelKNNCV2.fit(data)
modelKNNCV2.predict(dataT)
modelKNNCV2.printPlotSummary()

****** KNN CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.51    0.08
1    0.12    0.29

-----Best CV Parameters-----

n_neighbors = 10.00

...with the score = 0.79

###Decision Trees

In [10]:
modelTree = mdl.Tree(scale=True, max_depth=3, class_weight='balanced', random_state=1)
modelTree.fit(data)
modelTree.predict(dataT)
modelTree.printPlotSummary()

****** DECISION TREE ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.56    0.07
1    0.11    0.26

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.53    0.07
1    0.10    0.30

-----Feature Importance-----

Pclass      0.08
Age         0.12
Fare        0.13
EmbarkedC   0.00
EmbarkedS   0.00
Female      0.68
SibSpYes    0.00
ParchYes    0.00
dtype: float64

###Decision Trees CV

In [11]:
modelTreeCV = mdl.TreeCV(scale=True, class_weight='balanced', random_state=1)
modelTreeCV.fit(data)
modelTreeCV.predict(dataT)
modelTreeCV.printPlotSummary()

****** TREE CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

Pclass      0.08
Age         0.10
Fare        0.13
EmbarkedC   0.00
EmbarkedS   0.00
Female      0.69
SibSpYes    0.00
ParchYes    0.00
dtype: float64

-----Best CV Parameters-----

max_depth = 2.00

max_leaf_nodes = 6.00

...with the score = 0.81

###Random Forests

In [72]:
modelRandomForest = mdl.RandomForest(scale=True, n_estimators=128, max_features=None,
                                     max_depth=None, max_leaf_nodes=16, class_weight='balanced', random_state=1)
modelRandomForest.fit(data)
modelRandomForest.predict(dataT)
modelRandomForest.printPlotSummary()


****** RANDOM FOREST ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.56    0.06
1    0.06    0.31

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.53    0.07
1    0.09    0.32

-----Feature Importance-----

Pclass      0.08
Age         0.15
SibSp       0.03
Parch       0.00
Fare        0.19
EmbarkedC   0.01
EmbarkedS   0.01
Female      0.53
dtype: float64

###Boosted Trees

In [73]:
modelBoostedTree = mdl.BoostedTree(scale=True, n_estimators=256, loss='deviance', learning_rate=0.1, subsample=1.,
                                   max_features=None, max_depth=2, max_leaf_nodes=None, random_state=1,
                                   balanceWeights=True)
modelBoostedTree.fit(data)
modelBoostedTree.predict(dataT)
modelBoostedTree.printPlotSummary(cv=5)
modelBoostedTree.plotPartialDependence(features=('Age', 'Fare', 'Pclass', 
                                                 ('Age','Fare'),('Age','Pclass'),('Fare','Pclass'),
                                                 ('Female', 'Age'), ('Female','Pclass'),('Female','Fare')))

****** BOOSTED TREE ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

   0-pred  1-pred
0    0.57    0.06
1    0.05    0.32

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.51    0.08
1    0.08    0.32

-----Feature Importance-----

Pclass      0.06
Age         0.39
SibSp       0.06
Parch       0.01
Fare        0.38
EmbarkedC   0.00
EmbarkedS   0.01
Female      0.08
dtype: float64

###Boosted Trees CV

In [9]:
BoostedTreeCV = mdl.genModelCV(ModelClass=mdl.BoostedTree, grid={'learning_rate': np.exp2(np.arange(-5, 1., 1))})
modelBoostedTreeCV = BoostedTreeCV(cv=5, scale=True, n_estimators=256, loss='deviance', learning_rate=0.1, subsample=1.,
                                   max_features=None, max_depth=2, max_leaf_nodes=None, random_state=1,
                                   balanceWeights=True)
modelBoostedTreeCV.fit(data)
modelBoostedTreeCV.predict(dataT)
modelBoostedTreeCV.printPlotSummary(cv=5)
modelBoostedTreeCV.plotPartialDependence(features=('Age', 'Fare', 'Pclass',
                                                   ('Age', 'Fare'), ('Age', 'Pclass'), ('Fare', 'Pclass'),
                                                   ('Female', 'Age'), ('Female', 'Pclass'), ('Female', 'Fare')))


****** BOOSTED TREE CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

-----Feature Importance-----

Pclass      0.06
Age         0.37
Fare        0.43
EmbarkedC   0.01
EmbarkedS   0.01
Female      0.07
SibSpYes    0.02
ParchYes    0.02
dtype: float64

-----Best CV Parameters-----

learning_rate = 0.12

...with the score = 0.80   | avg = 0.79, std = 0.02

###Boosted Trees (XGBoost) - UNDER CONSTRUCTION

In [5]:
# modelBoostedTreeXG = mdl.BoostedTreeXGBoost(scale=True, n_estimators=128, loss='deviance', learning_rate=1.,
#                                             max_features=None, max_depth=2, max_leaf_nodes=None, random_state=1,
#                                             balanceWeights=True)
# modelBoostedTreeXG.fit(data)
# modelBoostedTreeXG.predict(dataT)
# modelBoostedTreeXG.printSummary()
# modelBoostedTreeXG.plotROC()


###SVM CV (Polynomial)

In [19]:
modelSVMCVPoly = mdl.SVMCV(cv=5, scale=True, kernel='poly', degree=4, class_weight='balanced', random_state=1)
modelSVMCVPoly.fit(data)
modelSVMCVPoly.predict(dataT)
modelSVMCVPoly.printPlotSummary()


*DEPRECIATED*  Use genModelCV instead

****** SVM CV (POLY): ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

...

   0-pred  1-pred
0    0.53    0.07
1    0.10    0.30

-----Best CV Parameters-----

C = 0.25

gamma = 0.25

...with the score = 0.80

###SVM CV (Radial)

In [6]:
modelSVMCVPoly = mdl.SVMCV(cv=5, scale=True, kernel='rbf', class_weight='balanced', random_state=1)
modelSVMCVPoly.fit(data)
modelSVMCVPoly.predict(dataT)
modelSVMCVPoly.printPlotSummary()


*DEPRECIATED*  Use genModelCV instead

****** SVM CV (RBF): ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

...

   0-pred  1-pred
0    0.51    0.09
1    0.10    0.30

-----Best CV Parameters-----

C = 4.00

gamma = 0.06

...with the score = 0.80

### Model Stacking Equal Weights

In [4]:
BaseKNNCV = mdl.genModelCV(mdl.KNN, grid={'n_neighbors': (5, 10, 20, 40)})\
    (cv=5, scale=False, weights='uniform')
BaseBoostedTreeCV = mdl.genModelCV(ModelClass=mdl.BoostedTree, grid={'learning_rate': np.exp2(np.arange(-5, 1., 1))}) \
    (cv=5, scale=False, n_estimators=256, loss='deviance', learning_rate=0.1, subsample=1.,
     max_features=None, max_depth=2, max_leaf_nodes=None, random_state=1, balanceWeights=True)
ModelVote = mdl.Vote(scale=True, Models=(('KNN',BaseKNNCV), ('BosteedTree', BaseBoostedTreeCV)), voting='hard', weights=None,
                     baseClassifiersInfo=False)
ModelVote.fit(data)
ModelVote.predict(dataT)
ModelVote.printPlotSummary(cv=5)

****** VOTE ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

<PROBABILITY IS NONE: SOME PERFORMANCE STATISTICS ARE NOT AVAILABLE>

...

-----Confusion (IS)-----

   0-pred  1-pred
0    0.61    0.02
1    0.11    0.27

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.55    0.04
1    0.12    0.28

<PROBABILITY IS NONE: ROC PLOT IS NOT AVAILABLE>

### Model Stacking & Selection CV

In [9]:
BaseLR = mdl.Logistic(fit_intercept=False, scale=False, C=10)
BaseKNNCV = mdl.genModelCV(mdl.KNN, grid={'n_neighbors': [5, 10, 20]})(cv=5, scale=False, weights='uniform')
BaseBoostedTreeCV = mdl.genModelCV(ModelClass=mdl.BoostedTree,
                                   grid={'learning_rate': [0.12, 0.06], 'subsample': [1., 0.8],
                                         'max_leaf_nodes': [4, 8]}) \
    (cv=5, scale=False, n_estimators=256, loss='deviance',
     max_features=None, random_state=1, balanceWeights=True)
BaseRandomForestCV = mdl.genModelCV(ModelClass=mdl.RandomForest, grid={'max_leaf_nodes': [8, 16, 32]}) \
    (cv=5, scale=False, n_estimators=256, max_features=None,
     max_depth=None, max_leaf_nodes=None, class_weight='balanced', random_state=1)
BaseSVMCV = mdl.genModelCV(ModelClass=mdl.SVM, grid={'degree': [2, 4], 'gamma': [0.06, 0.12, 0.25]}) \
    (cv=5, scale=False, C=0.25, kernel='poly', class_weight='balanced', random_state=1)
BaseLinearLocalCV = mdl.genModelCV(ModelClass=mdl.LogisticLinearLocal, grid={'bw': [0.25, 0.5, 1., 2.]}) \
    (cv=5, scale=False, reg_type='ll')
# BaseGAMCV = mdl.genModelCV(ModelClass=mdl.LogisticGAM, grid={'lam': np.exp2(np.arange(-4, 5, 2))}) \
#     (cv=5, scale=False, fit_intercept=False, n_splines=15)
BaseModelsCV = (('Logistic', BaseLR), ('KNN CV', BaseKNNCV), ('BoostedTree CV', BaseBoostedTreeCV),
                ('RandomForest CV', BaseRandomForestCV), ('SVM CV', BaseSVMCV), ('LinearLocal CV', BaseLinearLocalCV))


In [11]:
# Model Selection CV
weightsGrid = utmdl.weightsGrid(n=len(BaseModelsCV), step=1)
ModelVoteCV = mdl.VoteCV(cv=5, scale=True,
                         Models=BaseModelsCV,
                         voting='soft', weightsGrid=weightsGrid, baseClassifiersInfo=False)
ModelVoteCV.fit(data)
ModelVoteCV.predict(dataT)
ModelVoteCV.printPlotSummary(cv=5)


****** VOTE CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

-----Confusion (OOS)-----

   0-pred  1-pred
0    0.53    0.07
1    0.08    0.33

-----Best CV Parameters-----

BoostedTree CV = 1.00

...with the score = 0.81   | avg = 0.79, std = 0.01

In [12]:
# Model Stacking CV
weightsGrid = utmdl.weightsGrid(n=len(BaseModelsCV), step=0.1)
ModelVoteCV = mdl.VoteCV(cv=5, scale=True, Models=BaseModelsCV,
                         voting='soft', weightsGrid=weightsGrid, baseClassifiersInfo=False)
ModelVoteCV.fit(data)
ModelVoteCV.predict(dataT)
ModelVoteCV.printPlotSummary(cv=5)

****** VOTE CV ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

-----Best CV Parameters-----

-----Weights-----

KNN CV = 0.50

BoostedTree CV = 0.50

...with the score = 0.83   | avg = 0.81, std = 0.01

### Model Stacking CV with many base classifiers (slow)

In [25]:
# Model Stacking CV II
BaseModels = [['Logistic', mdl.Logistic(fit_intercept=False, scale=False, C=10)]]
for i, j in itertools.product([5, 10, 20], ['uniform']):
    BaseModels += [['KNN_{}_{}'.format(i, j), mdl.KNN(scale=False, n_neighbors=i, weights=j)]]
for i, j, k in itertools.product([0.12, 0.06], [1., 0.8], [4, 8]):
    BaseModels += [['BoostedTree_learn{}_sub{}_leaf{}'.format(i, j, k),
                    mdl.BoostedTree(scale=False, n_estimators=256, loss='deviance', learning_rate=i, subsample=j,
                                    max_features=None, max_depth=None, max_leaf_nodes=k, random_state=1,
                                    balanceWeights=True)]]
# for i in [8, 16, 32]:
#     BaseModels += [['RandomForest_leaf{}'.format(i),
#                     mdl.RandomForest(scale=False, n_estimators=256, max_features=None, max_depth=None,
#                                      max_leaf_nodes=i, class_weight='balanced', random_state=1)]]
# for i, j in itertools.product([2, 4], [0.06, 0.25]):
#     BaseModels += [['SVM_degree{}_gamma{}'.format(i, j),
#                     mdl.SVM(scale=False, C=0.25, kernel='poly', degree=i, gamma=j, class_weight='balanced',
#                             random_state=1)]]
# for i in [0.5, 1., 1.5,  2.5]:
#     BaseModels += [['LinearLocal_bw{}'.format(i),
#                     mdl.LogisticLinearLocal(scale=False, reg_type='ll', bw=i)]]
# for i in [2, 4, 8]:
#     BaseModels += [['LogisticGAM_lam{}'.format(i),
#                     mdl.LogisticGAM(scale=False, fit_intercept=False, n_splines=15, lam=i)]]
weightsGrid = utmdl.weightsGrid(n=len(BaseModels), step=0.2)
ModelVoteCV = mdl.VoteCV(cv=5, scale=True, Models=BaseModels, voting='soft', weightsGrid=weightsGrid)
ModelVoteCV.fit(data)
ModelVoteCV.predict(dataT)
ModelVoteCV.printPlotSummary(cv=5)


KeyboardInterrupt: 

### Model Stacking Regress

In [13]:
BaseModels = [['Logistic', mdl.Logistic(fit_intercept=False, scale=False, C=10)]]
for i, j in itertools.product([5, 10, 20], ['uniform']):
    BaseModels += [['KNN_{}_{}'.format(i, j), mdl.KNN(scale=False, n_neighbors=i, weights=j)]]
for i, j, k in itertools.product([0.12, 0.06], [1., 0.8], [4, 8]):
    BaseModels += [['BoostedTree_learn{}_sub{}_leaf{}'.format(i, j, k),
                    mdl.BoostedTree(scale=False, n_estimators=256, loss='deviance', learning_rate=i, subsample=j,
                                    max_features=None, max_depth=None, max_leaf_nodes=k, random_state=1,
                                    balanceWeights=True)]]
for i in [8, 16, 32]:
    BaseModels += [['RandomForest_leaf{}'.format(i),
                    mdl.RandomForest(scale=False, n_estimators=256, max_features=None, max_depth=None,
                                     max_leaf_nodes=i, class_weight='balanced', random_state=1)]]
for i, j in itertools.product([2, 4], [0.06, 0.12, 0.25]):
    BaseModels += [['SVM_degree{}_gamma{}'.format(i, j),
                    mdl.SVM(scale=False, C=0.25, kernel='poly', degree=i, gamma=j, class_weight='balanced',
                            random_state=1)]]
for i in [0.25, 0.5, 1., 2.]:
    BaseModels += [['LinearLocal_bw{}'.format(i),
                    mdl.LogisticLinearLocal(scale=False, reg_type='ll', bw=i)]]
# for i in [2, 4, 8]:
#     BaseModels += [['LogisticGAM_lam{}'.format(i),
#                     mdl.LogisticGAM(scale=False, fit_intercept=False, n_splines=15, lam=i)]]

In [14]:
# Model Stacking Regress (Square loss)
ModelVoteRegress = mdl.VoteRegress(cv=5, scale=True, Models=BaseModels, voting='soft', loss='square', 
                                   baseClassifiersInfo=False)
ModelVoteRegress.fit(data)
ModelVoteRegress.predict(dataT)
ModelVoteRegress.printPlotSummary(cv=5)

****** VOTE REGRESS ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

RandomForest_leaf8 = 0.13

SVM_degree4_gamma0.06 = 0.13

LinearLocal_bw1.0 = 0.09

LinearLocal_bw0.5 = 0.07

SVM_degree2_gamma0.25 = 0.01

In [15]:
# Model Stacking Regress (Deviance loss)
ModelVoteRegress = mdl.VoteRegress(cv=5, scale=True, Models=BaseModels, voting='soft', loss='deviance', baseClassifiersInfo=False)
ModelVoteRegress.fit(data)
ModelVoteRegress.predict(dataT)
ModelVoteRegress.printPlotSummary(cv=5)


****** VOTE REGRESS ******


-----Train and Test Sets-----

Sample Size (Train / Test): 623 / 268

Survived Rate (Train / Test): 0.37 / 0.41

-----Performance-----

...

BoostedTree_learn0.06_sub1.0_leaf8 = 0.00

SVM_degree2_gamma0.12 = 0.00

Logistic = 0.00

LinearLocal_bw2.0 = 0.00

KNN_10_uniform = 0.00

In [19]:
# Model Stacking Regress (Normalized deviance loss)
ModelVoteRegress = mdl.VoteRegress(cv=5, scale=True, Models=BaseModels, voting='soft', loss='devianceNormalized', baseClassifiersInfo=False)
ModelVoteRegress.fit(data)
ModelVoteRegress.predict(dataT)
ModelVoteRegress.printPlotSummary(cv=5)


KeyboardInterrupt: 